In [6]:
# To install talib follow instructions on this link - https://pypi.org/project/TA-Lib/
# Also install - pip install bokeh==3.1.0

from backtesting import Backtest, Strategy
from backtesting.test import GOOG
from backtesting.lib import crossover
import talib

In [7]:
print(GOOG)

              Open    High     Low   Close    Volume
2004-08-19  100.00  104.06   95.96  100.34  22351900
2004-08-20  101.01  109.08  100.50  108.31  11428600
2004-08-23  110.75  113.48  109.05  109.40   9137200
2004-08-24  111.24  111.60  103.57  104.87   7631300
2004-08-25  104.96  108.00  103.88  106.00   4598900
...            ...     ...     ...     ...       ...
2013-02-25  802.30  808.41  790.49  790.77   2303900
2013-02-26  795.00  795.95  784.40  790.13   2202500
2013-02-27  794.80  804.75  791.11  799.78   2026100
2013-02-28  801.10  806.99  801.03  801.20   2265800
2013-03-01  797.80  807.14  796.15  806.19   2175400

[2148 rows x 5 columns]


In [8]:
class RsiOscillator(Strategy):
    
    upper_bound=70
    lower_bound=30
    rsi_window=14
    
    def init(self):
        self.rsi=self.I(talib.RSI, self.data.Close, self.rsi_window)
    
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            self.buy()
            
bt = Backtest(GOOG, RsiOscillator, cash = 10_000)
stats = bt.run()
print(stats)

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   29.329609
Equity Final [$]                      15677.5
Equity Peak [$]                       15835.7
Return [%]                             56.775
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    5.416737
Volatility (Ann.) [%]               23.502544
Sharpe Ratio                         0.230475
Sortino Ratio                        0.386423
Calmar Ratio                         0.110499
Max. Drawdown [%]                  -49.020734
Avg. Drawdown [%]                   -6.135224
Max. Drawdown Duration     1603 days 00:00:00
Avg. Drawdown Duration       94 days 00:00:00
# Trades                                   10
Win Rate [%]                             80.0
Best Trade [%]                      21.397983
Worst Trade [%]                    -20.449092
Avg. Trade [%]                    

In [9]:
# 
bt.plot()
# pip install bokeh==3.1.0


D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


GridPlot(id='p2779', ...)

# With Optimization

In [14]:
class RsiOscillator(Strategy):
    
    upper_bound=70
    lower_bound=30
    rsi_window=14
    
    def init(self):
        self.rsi=self.I(talib.RSI, self.data.Close, self.rsi_window)
    
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            self.buy()
            
bt = Backtest(GOOG, RsiOscillator, cash = 10_000)

# We can choose our own optimization function. Here we have selected mazimizing Sharpe Ratio
stats = bt.optimize(
    upper_bound=range(50,85,5),
    lower_bound=range(10,45,5),
    rsi_window=range(10,30,2),
    maximize = "Sharpe Ratio",
    constraint = lambda param: param.upper_bound>param.lower_bound
)

#stats = bt.run()
print(stats)

D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   62.662942
Equity Final [$]                     45486.35
Equity Peak [$]                      45573.15
Return [%]                           354.8635
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   19.448742
Volatility (Ann.) [%]               32.629176
Sharpe Ratio                         0.596054
Sortino Ratio                        1.122836
Calmar Ratio                          0.35036
Max. Drawdown [%]                  -55.510789
Avg. Drawdown [%]                   -4.037504
Max. Drawdown Duration      548 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                                   17
Win Rate [%]                        76.470588
Best Trade [%]                      41.689994
Worst Trade [%]                    -27.637707
Avg. Trade [%]                    

In [13]:
bt.plot()

D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


GridPlot(id='p5205', ...)

# Evaluating other optimizing function

In [25]:
def optim_func(series):
    
    # Limit the number of trade
    if series["# Trades"]<10:
        return -1
    
    # How to make maximum profit by staying in the market for least amount of time
    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    
    upper_bound=70
    lower_bound=30
    rsi_window=14
    
    def init(self):
        self.rsi=self.I(talib.RSI, self.data.Close, self.rsi_window)
    
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            self.buy()
            
bt = Backtest(GOOG, RsiOscillator, cash = 10_000)

# We can choose our own optimization function. Here we have selected mazimizing Sharpe Ratio
stats = bt.optimize(
    upper_bound=range(50,85,5),
    lower_bound=range(10,45,5),
    rsi_window=range(10,30,2),
    maximize = optim_func,
    constraint = lambda param: param.upper_bound>param.lower_bound,
    max_tries=10
)

#stats = bt.run()
print(stats)

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   43.715084
Equity Final [$]                     29177.05
Equity Peak [$]                      29275.33
Return [%]                           191.7705
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   13.385606
Volatility (Ann.) [%]                28.06174
Sharpe Ratio                         0.477006
Sortino Ratio                        0.864783
Calmar Ratio                         0.249036
Max. Drawdown [%]                  -53.749743
Avg. Drawdown [%]                   -3.997825
Max. Drawdown Duration      787 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                   19
Win Rate [%]                        78.947368
Best Trade [%]                      42.917016
Worst Trade [%]                    -22.791549
Avg. Trade [%]                    

In [26]:
bt.plot()

D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


GridPlot(id='p12181', ...)

# Saving plot into a html format file

In [19]:

bt.plot(filename="plot01.html")

D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


GridPlot(id='p8844', ...)

In [20]:
print(stats["_strategy"])

RsiOscillator(upper_bound=50,lower_bound=25,rsi_window=10)


In [21]:
print(stats["_strategy"].upper_bound)

50


In [22]:
lower_bound=stats["_strategy"].lower_bound
lower_bound

25

In [24]:
bt.plot(filename=f'plots/plot-{lower_bound}.html')

D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
D:\AngelOneAPI\trading-sept24\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


GridPlot(id='p11068', ...)